In [ ]:
%matplotlib inline
import os
import pandas as pd
import pkg_resources as pkg
import pprint 

from oemof.solph import EnergySystem, Model
from oemof.tabular.facades import TYPEMAP
import oemof.tabular.tools.postprocessing as pp

from oemof.tabular import datapackage

## Setting Path to Datapackage 

The package comes with some example datapackage that you can use for testing. You can adapt this path to point to your datapackage. This scripts should work without any necessary additional changes. 

In [ ]:
name = "investment" # choose from ['dispatch', 'investment']

# path to directory with datapackage to load
datapackage_dir = pkg.resource_filename(
    "oemof.tabular", "examples/datapackages/{}".format(name)
)


For the results path we will use the name of the datapackage specified above and the home directory of the user.

In [ ]:
# create  path for results (we use the datapackage_dir to store results)
results_path = os.path.join(os.path.expanduser("~"), "oemof-results", name)
if not os.path.exists(results_path):
    os.makedirs(results_path)

## Setting `attributmap` and `typemap`

The two arguments allow for adjusting the datapackage reader your needs. The `attribute` map lets you specifiy to 
map column names of your datapackage resource (field names) to the facades. Take the following example: 

The `Load` facade requires the argument `amount`. If you like (for whatever reason) to use a different naming in your csv-file like `total_energy` you can do this be specifying the following: 

```
from oemof.tabular import facades as fc

...

attributemap = {
    fc.Load: {'amount', 'total_energy'}
}
```

So you can set the attribute map individually for all facades. However, we will use no mapping here for `attributemap` argument. 

In [ ]:
attributemap= {}

The `typemap` argument can be specified for your field (column name) `type` that specifcies which facade class should be used for instantiating the objects. We provide a default typemap. If you add your own facade classes, you **must** add these to typemap

```
typemap.update(
    {'my_new_class': my_class}
)
```



In [ ]:
typemap = TYPEMAP

# Look at current typemap
pprint.pprint(typemap)

## Create EnergySystem from Datapackage

Using the `.from_datapackge` method, creating your EnergySystem is straight forward

In [ ]:
# create energy system object
es = EnergySystem.from_datapackage(
    os.path.join(datapackage_dir, "datapackage.json"),
    attributemap=attributemap,
    typemap=typemap,
)
pprint.pprint(
    {n.label: n for n in es.nodes}
)

## Create Model 

This again is straight forward and just the way you would use the `oemof.solph` package. 

In [ ]:
# create model from energy system (this is just oemof.solph)
m = Model(es)

# if you want dual variables / shadow prices uncomment line below
# m.receive_duals()

# select solver 'gurobi', 'cplex', 'glpk' etc
m.solve("cbc")

# get the results from the the solved model(still oemof.solph)
m.results = m.results()

## Writing Results

The nice thing about `oemof.tabular.facades` classes is their data model that allows for writing results in a standardized way. If you want more indidual result postprocessing, you can use all functionalities of the `oemof.outputlib` as the results object is a standard object. 

In [ ]:
# now we use the write results method to write the results in oemof-tabular
# format
pp.write_results(m, results_path)



## Postprocessing Results

In [ ]:
print(listdir(results_path))

In [ ]:
bus = 'bus1'
path = os.path.join(results_path, ("").join([bus, '.csv'])) 
df = pd.read_csv(path, index_col=0, parse_dates=True)[1:48, 'ccgt'].plot()